<a href="https://colab.research.google.com/github/Vandan1009/python_1/blob/main/pspyk3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PySpark Practice Notebook
Experimenting further with Spark Dataframes

In [ ]:
#Create CSV files for the following Datasets

patients
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78

visits
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15

In [5]:
#Import necessary spark libraries
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=098fb06d5f5ad60199b5d893922f3d0a44795638834cc97c242f1a7e7427e460
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
#Create a SparkSession
from pyspark.sql import SparkSession


In [7]:
spark=SparkSession.builder.appName("notebook3").getOrCreate()

In [8]:
spark

In [9]:
#Create dataframes from CSV files using enforced Schema
#Schems Directive [Names as string, everything else as integers]
%%writefile names.csv
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78

Writing names.csv


In [10]:
%%writefile visit.csv
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15

Overwriting visit.csv


In [11]:
df1=spark.read.option('header','True').csv('visit.csv')

In [12]:
df=spark.read.option('header','True').csv('names.csv')

In [13]:
df.show()

+---------+---------+--------+---+
|patientId|firstName|lastName|age|
+---------+---------+--------+---+
|      101|    Alice|   Smith| 30|
|      102|      Bob| Johnson| 45|
|      103|  Charlie|Williams| 50|
|      104|     John|   Smith| 78|
+---------+---------+--------+---+



In [21]:
df1.show()

+-------+---------+-------------+
|visitId|patientId|visitDuration|
+-------+---------+-------------+
|      1|      101|           15|
|      2|      101|           30|
|      3|      102|           45|
|      4|      102|           30|
|      5|      104|           20|
|      6|      103|           60|
|      7|      103|           50|
|      8|      104|           45|
|      9|     NULL|           45|
|     10|      101|           30|
|     11|      103|           40|
|     12|     NULL|           30|
|     13|      104|           25|
|     15|      102|           15|
+-------+---------+-------------+



In [134]:
#Display all the valid visits
df1.filter(df1.patientId.isNotNull()).show()

+-------+---------+-------------+
|visitId|patientId|visitDuration|
+-------+---------+-------------+
|      1|      101|           15|
|      2|      101|           30|
|      3|      102|           45|
|      4|      102|           30|
|      5|      104|           20|
|      6|      103|           60|
|      7|      103|           50|
|      8|      104|           45|
|     10|      101|           30|
|     11|      103|           40|
|     13|      104|           25|
|     15|      102|           15|
+-------+---------+-------------+



In [32]:
from pyspark.sql.functions import *
import pyspark.sql.functions as f

In [55]:
#Fetch the total number of valid visits
# df2=df1.select('visitId').count()
# df2.show()
# df1.select('visitId').agg(count("visitId").alias("cnt")).show()
# df1.select('visit')
df1.select('visitId').agg({'visitId':'count'}).show()
df1.select('visitId').agg(count('visitId').alias('total')).show()

+--------------+
|count(visitId)|
+--------------+
|            14|
+--------------+

+-----+
|total|
+-----+
|   14|
+-----+



In [59]:
#Get the total hours of patient visit
# df2=df1.filter('patientId').isNotNull()
# df2.show()
#df1.groupBy('patientId').agg({'visitDuration':'sum'}).dropna().show()
df1.groupby('patientId').agg(max('visitDuration').alias('max_hou')).dropna().show()

+---------+-------+
|patientId|max_hou|
+---------+-------+
|      101|     30|
|      102|     45|
|      103|     60|
|      104|     45|
+---------+-------+



In [137]:
#Fetch the top two most visited patients on the basis of occurances
from pyspark.sql.functions import desc
# df5=df1.groupBy('patientId').agg({'visitId':'max'}).dropna()
# df5.orderBy(col('max(visitId)').desc()) .limit(2).show()
a=df1.groupBy('patientId').agg(max('visitId').alias('ha'))
a.orderBy(a.ha.desc()).limit(2).show()


#orderBy(desc("ha")).limit(2).show()
#df1.groupBy('patientId').agg(max('visitId')).orderBy(col('max(visitId)').desc()).limit(2).show()
#df1.groupBy('patientId').agg(max('visitId').alias('vi')).orderBy(col('vi').desc()).limit(2).show()

AttributeError: 'GroupedData' object has no attribute 'dropna'

In [35]:
df.withColumn('fullname',f.concat_ws(' ','firstName','lastName')).show()
# 'fristName' corrected to 'firstName'


+---------+---------+--------+---+----------------+
|patientId|firstName|lastName|age|        fullname|
+---------+---------+--------+---+----------------+
|      101|    Alice|   Smith| 30|     Alice Smith|
|      102|      Bob| Johnson| 45|     Bob Johnson|
|      103|  Charlie|Williams| 50|Charlie Williams|
|      104|     John|   Smith| 78|      John Smith|
+---------+---------+--------+---+----------------+



In [96]:
from pyspark.sql.functions import concat_ws

In [100]:
import pyspark.sql.functions as f

In [106]:
#Add a new column patientFullName & display complete dataframe
b=df.withColumn('fullname',f.concat_ws(' ','firstName','lastName'))
b.show()

+---------+---------+--------+---+----------------+
|patientId|firstName|lastName|age|        fullname|
+---------+---------+--------+---+----------------+
|      101|    Alice|   Smith| 30|     Alice Smith|
|      102|      Bob| Johnson| 45|     Bob Johnson|
|      103|  Charlie|Williams| 50|Charlie Williams|
|      104|     John|   Smith| 78|      John Smith|
+---------+---------+--------+---+----------------+



In [139]:
#Display the name of patients with maximum visiting hours
# a=b.join(df1,on='patientId',how='inner')
# a.show()

a.groupBy('fullname').agg(max('visitDuration')).show()

+----------------+------------------+
|        fullname|max(visitDuration)|
+----------------+------------------+
|     Alice Smith|                30|
|     Bob Johnson|                45|
|Charlie Williams|                60|
|      John Smith|                45|
+----------------+------------------+



In [118]:
a.groupBy('fullname').agg(max('visitDuration')).show()

+----------------+------------------+
|        fullname|max(visitDuration)|
+----------------+------------------+
|     Alice Smith|                30|
|     Bob Johnson|                45|
|Charlie Williams|                60|
|      John Smith|                45|
+----------------+------------------+



In [129]:
#Find the patient with maximum visiting hours in a single visit
#df1.groupBy('patientId').agg(max('visitDuration')).dropna().show()

df1.filter(col('patientId').isNotNull()).agg(max('visitDuration')).show()
df1.filter(col('patientId').isNotNull()).groupBy('patientId',"visitDuration").count().sort('count',ascending=False).show()

+------------------+
|max(visitDuration)|
+------------------+
|                60|
+------------------+

+---------+-------------+-----+
|patientId|visitDuration|count|
+---------+-------------+-----+
|      101|           30|    2|
|      103|           40|    1|
|      104|           25|    1|
|      104|           20|    1|
|      103|           50|    1|
|      102|           15|    1|
|      102|           30|    1|
|      101|           15|    1|
|      102|           45|    1|
|      104|           45|    1|
|      103|           60|    1|
+---------+-------------+-----+



In [141]:
#Show the patient info according to age-seniority
df.sort('age',ascending=False).show()

+---------+---------+--------+---+
|patientId|firstName|lastName|age|
+---------+---------+--------+---+
|      104|     John|   Smith| 78|
|      103|  Charlie|Williams| 50|
|      102|      Bob| Johnson| 45|
|      101|    Alice|   Smith| 30|
+---------+---------+--------+---+

